#Apriori
```
Versión   : Actividad 3
Autor     : Daniel Eduardo Sarco Jacinto
Lugar     : Cusco, Perú, 2021.
Proposito : Implementación del algoritmo de Apriori
```

**Librerias a usar**

In [3]:
import numpy as np

#1. Implementar el algoritmo Apriori

Se debe implementar el algoritmo según lo visto en clases.
Para estructurar mejor tu código, deberás implementar las siguientes funciones:


1.   get_frequent_itemsets(playlists, min_support): Recibe la esctructura de datos que contiene a las playlists y retorna una estructura con los itemsets frecuentes, bajo un umbral mínimo de confianza.
2.   generate_association_rules(frequent_itemsets, confidence = 0, lift = 0): Recibe los itemsets frecuentes generados por la función anterior y retorna las
reglas de asociación. Se le puede entregar umbrales de confianza o lift para las
reglas que se retornarán. Por ejemplo, si se llama esta función con los ar- gumentos confidence = 0.5 y lift = 1.2, se espera que se retornen aquellas reglas que cumplan con una confianza ≥ 0.5 y un lift ≥ 1.2. Puedes utilizar las estructuras de datos que estimes convenientes para representar

la información, tanto de las playlists, los itemsets frecuentes y las reglas de asociación. Estas pueden ser lista de listas, numpy arrays o pandas DataFrames. Sin embargo, esto debe estar claramente documentado en el mismo notebook.

**Motodos Necesarios para el algoritmo apriori (metodos propios)**

In [40]:
"""
Modulo para cargar el archivo spotify.npy con numpy
"""
def Leer_archivo_npy(nombre_archivo_npy):
  data = np.load(nombre_archivo_npy,allow_pickle=True)#leemos el archivo
  Lista_data=data.tolist()#Convertimo en en lista el Data leido
  return Lista_data
"""
Metodo para verificar si un conjunto pertenece al un arreglo de conjunto
"""
def Verificar_conjunto_contenido_conjuntos(Conjuntos,ConjuntoEvaluar):
  for i in Conjuntos:                      # Recorrer cada conjunto
    ConjuntoEvaluar=set(ConjuntoEvaluar)   # Convertir en conjunto el ConjuntoEvaluado
    if(ConjuntoEvaluar.issubset(set(i))):  # Verificar pertenecia de conjuntos
      return True                          # retornar verdadero si pertence
  return False                             # retornar falso si no pertenece
"""
Metodo para realizar un inner join entre 2 conjuntos y el parametro Tam para restringir que solo se quiere un arreglo de Tam elementos
"""
def InnerJoinConjuntos(Conjunto1,Conjunto2,Tam):
  Conjunto_resultante=[]                          # Arreglo donde almacenaremos el nuevo conjunto generado
  for Ele1 in Conjunto1:                          # Recorremos los elemento del Conjunto1
    for Ele2 in Conjunto2:                        # Recorremos los elemento del Conjunto2
      if(Ele1!=Ele2):                             # Verificar que los elementos del conjunto 1 y 2 no sean iguales 
          Arreglo=set()                           # Definimos un nuevo arreglo
          Ele1.sort()                             # Ordenamos los elementos del conjunto 1
          Ele2.sort()                             # Ordenamos los elementos del conjunto 2
          ArregloAux1=set(Ele1[0:len(Ele1)-1])    # Almacenamos los elementos del conjunto 1 pero sin considerar el ultimo elemento
          ArregloAux2=set(Ele2[0:len(Ele2)-1])    # Almacenamos los elementos del conjunto 2 pero sin considerar el ultimo elemento
          if(ArregloAux1.issubset(ArregloAux2)):  # Verificar que los arreglo definidos esten incluido mutuamente
            Arreglo|=set(Ele1)                    # Union de los conjuntos (Arreglo) y el elemento del conjunto 1
            Arreglo|=set(Ele2)                    # Union de los conjuntos (Arreglo) y el elemento del conjunto 2
            Arreglo=list(Arreglo)                 # Lo Convertimos en lista el conjunto(Arreglo)
            Arreglo.sort()                        # Ordenenamos de menor a mayor el conjunto(Arreglo)
            if(Tam==len(Arreglo) and not(Verificar_conjunto_contenido_conjuntos(Conjunto_resultante,Arreglo))): # Verficar que este conjunto no se repita y tambien que su tamaño sea igual
              Conjunto_resultante.append(Arreglo) # Agregar el conjunto(Arreglo) a nuestro Conjunto_resultante
  return Conjunto_resultante                      # Retornamos el Conjunto_resultante

"""
Metodo para llevar un arreglo unidimensional en un arreglo bidimensional
""" 
def Convertir_unidimensional_bidimensional(Arreglo):
  NuevoArreglo=[]             # Definir arrego
  for i in Arreglo:           # Recorrer el Arreglo
    NuevoArreglo.append([i])  # Agregar el elemento en el nuevo Arreglo
  return NuevoArreglo         # Retornar NuevoArreglo

"""
Metodo para obtener los posibles candidatos de un data en general
""" 
def Iteraciones_candidatos(playlists_arreglo,playlists_conjunto,min_support,playlists_candidatos_posibles,playlist_frecuencias_candidatos,nIteracion):
  #Instrucciones la cual me ayudan a calcular la ocurrencia de las canciones
  playlists_Frecuencia=[0]*len(playlists_conjunto)         # Definimos un arreglo donde se almacenara las frecuecias o Ocurrencias de los elementos de playlists_conjunto donde se inicializa en 0 
  for i in range(len(playlists_arreglo)):                  # Recorremos los playlists de la plataforma spotify
    playlist=set(playlists_arreglo[i])                     # Obtener un playlist de la plataforma spotify
    for j in range(len(playlists_conjunto)):               # Recorremos las canciones del playlists_conjunto
      Conjunto_playlist=set(playlists_conjunto[j])         # Cada cancion se convierte en un conjunto con el objetivo de realizar operaciones con conjunto
      if(Conjunto_playlist.issubset(playlist)):            # Verificar que esa cancion extraida pertenezca al data en general(playlists_arreglo)
        playlists_Frecuencia[j]=playlists_Frecuencia[j]+1  # Aumentamos su frecuencia de la cancion extraida
  
  #Estraer solo las canciones que cumple las condiciones
  playlist_frecuentes=[]                                   # Definir un arreglo para agregar los candidatos posibles en cada interacion
  playlist_frecuencias=[]                                  # Definir un arreglo para agregar las ocurrencias de los candidatos posibles en cada interacion
  for i in range(len(playlists_Frecuencia)):               # Recorremos las frecuencia ya definidas
    if((playlists_Frecuencia[i]/len(playlists_arreglo))>=min_support):            # Verificar que cada ocurrencia sea igual o mayor que el min_support
      playlist_frecuentes.append(playlists_conjunto[i])    # Agregamos las canciones candidatas
      playlist_frecuencias.append(playlists_Frecuencia[i]) # Agregamos las las fecuencias correnpodinetes a cada cancion candidata

  #Verificar si vamos a llevar realizar una nueva interacion hasta el final
  if(len(playlist_frecuentes)<=0):                         # Si no hay candidatos significa que terminamos de iterar
    return [],[]                                           # retornamos nulo por que es el final de la recursividad
  else:                                                    # En caso de haber candidatos que cumpla la condicion se procede iterar uno nuevo
    playlists_candidatos_posibles.append(playlist_frecuentes)  # Agregar el playlist_frecuentes en nuestro arreglo playlists_candidatos_posibles
    playlist_frecuencias_candidatos.append(playlist_frecuencias) # Agregar el playlist_frecuencias en nuestro arreglo playlist_frecuencias_candidatos
    playlists_conjunto=InnerJoinConjuntos(playlist_frecuentes,playlist_frecuentes,nIteracion+2) # Realizamos un inner join entre playlist_frecuentes y playlist_frecuentes y incrementamos la interacion
    Nueva_Iteracion,Nueva_frecuencias=Iteraciones_candidatos(playlists_arreglo,playlists_conjunto,min_support,playlists_candidatos_posibles,playlist_frecuencias_candidatos,nIteracion+1) # Llamos al mismo metodo de nuevo para el nuevo conjunto que es el playlists_conjunto
    if(Nueva_Iteracion==[] or Nueva_frecuencias==[]): #Verificar que no nos retorne vacios
      return playlists_candidatos_posibles,playlist_frecuencias_candidatos # retornar playlists_candidatos_posibles y playlist_frecuencias_candidatos
    else:
      return Nueva_Iteracion,Nueva_frecuencias # retornar Nueva_Iteracion y Nueva_frecuencias
"""
Metodo para buscar el soporte(Ocurrencia/cantidad playlists) de de un conjunto ingresado como prarametro 
"""     
def Buscar_Soporte_Itemets(frequent_itemsets,Soporte,ConjuntoEvaluar):                  # Frequent_itemsets[0]: lista de canciones candidatas ,  frequent_itemsets[1]: Frecuencias de las canciones candidatas  frequent_itemsets[2]: Cantidad de playlists
  for i in range(len(frequent_itemsets[0])):                                            # Recorrer el conjunto de canciones candidatas(Frequent_itemsets[0]) 
    Fre=frequent_itemsets[0][i]                                                         # Extraer El conjunto de canciones de frequent_itemsets[0]
    for j in range(len(Fre)):                                                           # Recorrer cada cancion de Fre
      Musicas=Fre[j]                                                                    # Extraer la cancion de Fre
      Musicas=set(Musicas)                                                              # Convertir la musica en conjunto
      if(ConjuntoEvaluar.issubset(Musicas) and (len(ConjuntoEvaluar)==len(Musicas))):   # Verificar si encontramos el soporte correspondiente a ConjuntoEvaluar
        return Soporte[i][j]                                                            # retorna el soporte encontrado
  return -1                                                                             # Retornar -1 si no encontro el soprote correspondiente

"""
Metodo para verificar que las reglas que se a generado proceda a validar con respecto al lift y confidence
"""   
def Verificar_reglas_lift_confidence(frequent_itemsets,Soporte,Reglas_itemsets,lift,confidence):
  Nuevas_Reglas=[]
  ResultadoFinal=[]                                                                     # Arreglo donde se almacenara las nuevas reglas con cumplen la condicion de lift y confidence
  for Regla in Reglas_itemsets:                                                         # Recorrer las reglas que contiene Reglas_itemsets
    ArregloIz=Regla[0]                                                                  # Canciones de lado izquierdo (izquierdo  => derecho)
    ArregloDe=Regla[1]                                                                  # Canciones de lado derecho (izquierdo  => derecho)
    ArregloIz=set(ArregloIz)                                                            # Convetir en conjunto
    ArregloDe=set(ArregloDe)                                                            # Convetir en conjunto

    # Realizar operacion para verificar si comple con lift
    Numerador=Buscar_Soporte_Itemets(frequent_itemsets,Soporte,ArregloIz | ArregloDe)   
    Denominador=Buscar_Soporte_Itemets(frequent_itemsets,Soporte,ArregloIz)*Buscar_Soporte_Itemets(frequent_itemsets,Soporte,ArregloDe)
    Resultado=Numerador/Denominador
    if(Resultado>=lift):#verificar que cumpla la condicion
      # Realizar operacion para verificar si comple con confidence
      Numerador=Buscar_Soporte_Itemets(frequent_itemsets,Soporte,ArregloIz | ArregloDe)
      Denominador=Buscar_Soporte_Itemets(frequent_itemsets,Soporte,ArregloIz)
      Resultado=Numerador/Denominador
      if(Resultado>=confidence):#Verificar que se cumpla
        Nuevas_Reglas.append(Regla) # Si se cumplio las 2 condiciones entonces esta reglas es valida por lo que se agregar en Nuevas_Reglas
        ResultadoFinal.append(Resultado)
  return Nuevas_Reglas,ResultadoFinal # Retornar la reglas nuevas validadas

"""
Metodo mostar las reglas finales
"""  
def MortrarReglas(Reglas_lift_confidence):
  for i in Reglas_lift_confidence:
    for j in i:
      print(j[0]," => ",j[1])

"""
Metodos Para realizar una combinatoria en un arreglo
"""  
def potencia(pValor):
    if len(pValor) == 0:
        return [[]]
    r = potencia(pValor[:-1])
    return r + [s + [pValor[-1]] for s in r]
def combinaciones(A, pValor):
    return [s for s in potencia(A) if len(s) == pValor]
"""
Metodos extraer el complemento de un subArreglo de un Arrglo (Ejemplos: ([1,2,3],[3]) = [1,2] )
"""     
def Get_Arreglo_Sobrante(Arreglo,SubArreglo):
  ArregloResultante=[]
  for i in Arreglo:
    if(not(i in SubArreglo)):
      ArregloResultante.append(i)
  return ArregloResultante
"""
Metodos generar ya las reglas de un solo itemsets
"""     
def Generar_Reglas(A):
  Reglas=[]
  for i in range(len(A)-1):
    Combinacion=combinaciones(A, i+1)
    for j in range(len(Combinacion)):
      CombinacionRestante=combinaciones(Get_Arreglo_Sobrante(A,Combinacion[j]),(len(A)-(i+1)))
      Reglas.append([Combinacion[j],CombinacionRestante[0]])
  return Reglas

**Motodos Principales para el algoritmo apriori (metodos propios)**

In [34]:
"""
Recibe la esctructura de datos que contiene a las playlists y retorna una estructura con los itemsets fre-
cuentes, bajo un umbral mínimo de confianza.
"""
def get_frequent_itemsets(playlists, min_support):
  playlists_conjunto=set()                            # Múltiples playlists de la plataforma spotify
  playlists_arreglo=[]                                # Arreglo de itemsets de playlists
  for i in playlists:                                 # Recorrer múltiples playlists de la plataforma spotify
    playlists_conjunto|=set(playlists[i])             # Agregar un playlists en el conjunto "playlists_conjunto" para obtener una lista de canciones sin repetir
    playlists_arreglo.append(set(playlists[i]))       # Agregar un playlists en el arreglo "playlists_arreglo"
  Cantidad_playlists=len(playlists_arreglo)           # Cantidad de playlists que existen en la plataforma spotify
  playlists_candidatos_posibles=[]
  playlist_frecuencias_candidatos=[]
  playlists_candidatos_posibles=Iteraciones_candidatos(playlists_arreglo,Convertir_unidimensional_bidimensional(list(playlists_conjunto)),min_support,playlists_candidatos_posibles,playlist_frecuencias_candidatos,0)
  return playlists_candidatos_posibles
"""
Recibe los itemsets frecuentes generados por la función anterior y retorna las
reglas de asociación. Se le puede entregar umbrales de confianza o lift para las
reglas que se retornarán. Por ejemplo, si se llama esta función con los ar- gumentos
confidence = 0.5 y lift = 1.2, se espera que se retornen aquellas reglas que
cumplan con una confianza ≥ 0.5 y un lift ≥ 1.2.
"""
def generate_association_rules(frequent_itemsets, confidence = 0, lift = 0):
  #Determinando el soporte de las frecuencias de los frequent_itemsets
  Soporte=[]
  for i in frequent_itemsets[1]:
    SoporteAux=[]
    for j in range(len(i)):
      SoporteAux.append(i[j]/frequent_itemsets[2])
    Soporte.append(SoporteAux)
  items_frecuentes=frequent_itemsets[0][len(frequent_itemsets)-1]
  Reglas_en_general=[]
  Resultado_association_rules=[]
  for i in range(len(items_frecuentes)):
    Reglas_itemsets=Generar_Reglas(items_frecuentes[i])
    Reglas_lift_confidence,ResultadoFinal=Verificar_reglas_lift_confidence(frequent_itemsets,Soporte,Reglas_itemsets,lift,confidence)
    Reglas_en_general.append(Reglas_lift_confidence)
    Resultado_association_rules.append(ResultadoFinal)
  return Reglas_en_general,Resultado_association_rules
         

In [35]:
def Algoritmo_Apriori():
  playlists = Leer_archivo_npy("spotify.npy")#Leer el archivo .npy
  playlists_candidatos_posibles,FrecuencasCandidatas=get_frequent_itemsets(playlists,0.0001)
  Reglas_asociacion,Resultados=generate_association_rules([playlists_candidatos_posibles,FrecuencasCandidatas,len(playlists)], confidence = 0.5, lift =1)
  return Reglas_asociacion,Resultados

#2. Aplicar el algoritmo y obtener reglas de asociación

**Deberás aplicar el algoritmo implementado en la base de datos entregada y filtrar las
mejores 10 reglas de asociación de acuerdo a dos criterios de calidad definidos por
ti. Deberás explicar por qué elegiste estos criterios.**

>La siguiente instruccion es para obtener las reglas de asociacion con sus respectivos resutlados(umbral de confianza) de dataset spotify.npy
(NOTA: su tiempo de ejecucion es mas de 1-3 horas)

In [ ]:
Reglas_asociacion,Resultados=Algoritmo_Apriori()

>Como el algoritmo demora demasiado(es dificil dejarlo en computador prendido compilando ya que produce un en el servidor de tiempo excedido) y no puedo esta ejecutandolo para ver si el algoirtmo se implemento exitosamente por lo se procedio a tomar como referencias los siguientes ejemplos de internet para su implementacion exitosa.


1.   https://es.myservername.com/apriori-algorithm-data-mining
2.   https://www.cienciadedatos.net/documentos/43_reglas_de_asociacion

>Teniendo estos ejemplos se procedio a usar uno de ellos que a continuacion son las siguientes instrucciones



In [44]:
#de duplico el metodo de get_frequent_itemsets por que como es un arreglo definido algunos parametro cambia
def get_frequent_itemsets_2(playlists, min_support):
  playlists_conjunto=set()                            # Múltiples playlists de la plataforma spotify
  playlists_arreglo=[]                                # Arreglo de itemsets de playlists
  for i in playlists:                                 # Recorrer múltiples playlists de la plataforma spotify
    playlists_conjunto|=set(i)                        # Agregar un playlists en el conjunto "playlists_conjunto" para obtener una lista de canciones sin repetir
    playlists_arreglo.append(set(i))                  # Agregar un playlists en el arreglo "playlists_arreglo"
  Cantidad_playlists=len(playlists_arreglo)           # Cantidad de playlists que existen en la plataforma spotify
  playlists_candidatos_posibles=[]
  playlist_frecuencias_candidatos=[]
  playlists_candidatos_posibles=Iteraciones_candidatos(playlists_arreglo,Convertir_unidimensional_bidimensional(list(playlists_conjunto)),min_support,playlists_candidatos_posibles,playlist_frecuencias_candidatos,0)
  return playlists_candidatos_posibles
def Algoritmo_Apriori_2():
  playlists = [["I2","I1","I3"],
               ["I2","I3","I4"],
               ["I4","I5"],
               ["I1","I2","I4"],
               ["I1","I2","I3","I5"],
               ["I1","I2","I3","I4"]]
  playlists_candidatos_posibles,FrecuencasCandidatas=get_frequent_itemsets_2(playlists,eval("3/6"))
  Reglas_asociacion,Resultados=generate_association_rules([playlists_candidatos_posibles,FrecuencasCandidatas,len(playlists)], confidence = 0, lift =0)
  return Reglas_asociacion,Resultados

Reglas,Resultado=Algoritmo_Apriori_2()
print("==> Reglas generadas ")
MortrarReglas(Reglas)
print("==> umbral de confianza respectivamente a cada regla ")
print(Resultado)

==> Reglas generadas 
['I1']  =>  ['I2', 'I3']
['I2']  =>  ['I1', 'I3']
['I3']  =>  ['I1', 'I2']
['I1', 'I2']  =>  ['I3']
['I1', 'I3']  =>  ['I2']
['I2', 'I3']  =>  ['I1']
==> umbral de confianza respectivamente a cada regla 
[[0.75, 0.6, 0.75, 0.75, 1.0, 0.75]]


#3. Explicar las reglas obtenidas

**Deberás seleccionar 4 reglas y comentar su calidad de acuerdo a los diferentes indicadores disponibles (support, confidence y lift). Además, según el género y/o el artista de las canciones (que puedes buscar según el nombre de la canción) deberás darle una breve interpretación a las reglas.**

Al ser muchos datos,la demora de copilacion del algoritmo apriori es muy lenta por lo que no es dificil estar compilado cada rato por lo que de decidio tomar como referencia un ejemplo del internet la cual este algoritmo implementado lo compila exitosamente

>Explicacion: Tomando como referencia ejemplos del internet y asociandolo como si fueran canciones su interpretacion es la siguiente



1. ['I1']  =>  ['I2', 'I3'] :Indica que los que escuchan la cancion I1 es probable que tambien escuchen las canciones I2 y I3
2. ['I2']  =>  ['I1', 'I3'] :Indica que los que escuchan la cancion I2 es probable que tambien escuchen las canciones I1 y I3
3. ['I3']  =>  ['I1', 'I2'] :Indica que los que escuchan la cancion I3 es probable que tambien escuchen las canciones I1 y I2
4. ['I1', 'I2']  =>  ['I3'] :Indica que los que escuchan la cancion I1 y T2 es probable que tambien escuchen la cancion I3
